# Big Data Analytics

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [ ]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [ ]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [ ]:
print_count(trips)

Row count: 2,898,033


In [ ]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [ ]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [ ]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [ ]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [ ]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

### Task 1.1 - Explanation
In the below cell, I used pipelined to filter the values of trip distance greater than zero (as trip without distance shouldn't be considered), the values of passenger count equal to zero and the values of total amount equal to zero (retain where total_amount>0) and at last select the relevant dataset required further for processing.

In [ ]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    df = (
        df.filter((df.trip_distance > 0)) #Remove "0 distance"
          .filter(~((df.passenger_count == 0) & (df.total_amount ==0))) #Remove 'no passengers' records and retain where total_amount>0
          .select(['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']) #Selecting relevant dataset
    )
    return df  

In [ ]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

# # uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 2,858,164


###Task 1.2 - Explanation
The below code removes outliers from a PySpark dataframe based on the median and MAD of two columns: total_amount and trip_distance. The median is calculated using the percentile_approx function with a specified accuracy of 1000 or 100 depending on the column. The absolute deviation from the median (delta_median) is calculated for each column. The MAD is calculated as the median of delta_median multiplied by a constant scaling factor of 1.4826. Z-scores are calculated for each column by subtracting the median from each value and dividing by the MAD. Finally, rows with z-scores exceeding 3.5 in absolute value are filtered out, and the resulting dataframe is returned with the desired columns.

In [ ]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    # calculate median and MAD of total_amount
    median_total_amount = df.agg(pf.percentile_approx("total_amount", 0.5, pf.lit(1000))).collect()[0][0]
    delta_median_total_amount_abs = pf.abs(df.total_amount - median_total_amount)
    mad_total_amount = df.agg(pf.percentile_approx(delta_median_total_amount_abs, 0.5, pf.lit(1000))).collect()[0][0] * 1.4826
    
    # calculate median and MAD of trip_distance
    median_trip_distance = df.agg(pf.percentile_approx("trip_distance", 0.5, pf.lit(1000))).collect()[0][0]
    delta_median_trip_distance_abs = pf.abs(df.trip_distance - median_trip_distance)
    mad_trip_distance = df.agg(pf.percentile_approx(delta_median_trip_distance_abs, 0.5, pf.lit(1000))).collect()[0][0] * 1.4826
    
    # calculate z-scores using MAD
    z_score_total_amount = ((df.total_amount - median_total_amount) / mad_total_amount)
    z_score_trip_distance = ((df.trip_distance - median_trip_distance) / mad_trip_distance)
    
    # filter outliers
    df = df.filter((pf.abs(z_score_total_amount) <= 3.5) & (pf.abs(z_score_trip_distance) <= 3.5))
    
    return df

In [ ]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 2,495,632


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [ ]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

### Task 2.1 - Explanation

The below code is adding more information to the taxi trip data by including the names of the zones where the trips started and ended. This will help in analyzing the trips and understanding which zones are more popular and how different service zones are being used. Then, I returned the dataframe with the updated columns

In [ ]:
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    df = (
        df.alias('d')
          .join(zones_df.alias('z1'), pf.col('d.PULocationID') == pf.col('z1.LocationID'), 'left')
          .join(zones_df.alias('z2'), pf.col('d.DOLocationID') == pf.col('z2.LocationID'), 'left')
          .select(
              'd.*',
              pf.col('z1.Zone').alias('start_zone'),
              pf.col('z1.service_zone').alias('start_service_zone'),
              pf.col('z2.Zone').alias('end_zone'),
              pf.col('z2.service_zone').alias('end_service_zone')
          )
    )
    
    return df


In [ ]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 2,495,632


## Task 2.2 - Explanation

The below function is used to calculated unit profitability by dividing the total amount by trip distance and stored in a new column 'unit_profitability'. Then, I returned the dataframe with the updated columns

In [ ]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn('unit_profitabilty',
                       pf.when(df.trip_distance==0, 0)
                         .otherwise(df.total_amount/df.trip_distance)
                      )
    return df

In [ ]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
# display(trips_22.take(10))

Row count: 2,495,632


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [ ]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

### Task 3.1 - Explanation

The below function groups the data by pick-up and drop-off location IDs, and calculates aggregated information about trips between each pair of locations, such as average unit profitability, number of trips, and total number of passengers. It returns a new DataFrame with the summarized information.

In [ ]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2  
    df = (
            df.groupBy(['PULocationID', 'DOLocationID'])
              .agg(
                  pf.mean('unit_profitabilty').alias('average_unit_profit'),
                  pf.count('trip_distance').alias('trips_count'),
                  pf.sum('passenger_count').alias('total_passengers'))
    )
    return df

In [ ]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 13,699


PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers
90,231,7.432778991914051,1123,1576.0
87,33,6.817160729833857,105,145.0
90,142,6.873745233124353,907,1276.0
114,100,7.480600705110231,465,705.0
234,144,9.131779529653814,1487,2186.0
148,262,4.840744491138079,280,415.0
246,249,8.820105729239454,1150,1713.0
170,179,5.138416672570028,79,102.0
48,232,5.3943422140115045,362,527.0
142,144,5.766557628964919,351,492.0


### Task 3.2 - Explanation

The implementation consists of the following four functions:

t32_summarise_zones_pairs(df, zones_df = zone_names): This function summarizes the trips data by pickup zone by joining the trips data with a zones data frame and grouping the resulting data frame by pickup zone. It then calculates the total trips, average unit profit, and total passengers for each pickup zone. The resulting data frame is returned.

t32_top10_trips(df_zones): This function takes the output of the previous function as input and returns the top 10 ranked zones based on trip volume (total trips), sorted in descending order. It uses the 'orderBy' method to sort the dataframe by 'total_trips' in descending order, and then uses the 'limit' method to restrict the result to the top 10 rows. Finally, it selects only the 'pickup_zone' and 'total_trips' columns.

t32_top10_profit(df_zones): This function takes the output of the first function as input and returns the top 10 ranked zones based on average unit profit, sorted in descending order. It uses the 'orderBy' method to sort the dataframe by 'average_unit_profit' in descending order, and then uses the 'limit' method to restrict the result to the top 10 rows. Finally, it selects only the 'pickup_zone' and 'average_unit_profit' columns.

t32_top10_passenger(df_zones): This function takes the output of the first function as input and returns the top 10 ranked zones based on passenger volume (total passengers), sorted in descending order. It uses the 'orderBy' method to sort the dataframe by 'total_passengers' in descending order, and then uses the 'limit' method to restrict the result to the top 10 rows. Finally, it selects only the 'pickup_zone' and 'total_passengers' columns.

In [ ]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)

def t32_summarise_zones_pairs(df, zones_df = zone_names):
    df = (
        df.join(zones_df, df.PULocationID == zones_df.LocationID, 'left')
          .withColumnRenamed('Zone', 'pickup_zone')
          .withColumnRenamed('service_zone', 'pickup_service_zone')
          .groupBy('pickup_zone')
          .agg(
              pf.sum('trips_count').alias('total_trips'),
              pf.avg('average_unit_profit').alias('average_unit_profit'),
              pf.sum('total_passengers').alias('total_passengers')
          )
    )
    return df


# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    window_spec = ps.window.Window.orderBy(pf.desc('total_trips'))
    t32_top10_trips = (
        df_zones.withColumn('rank', pf.rank().over(window_spec))
                .select('rank', 'pickup_zone', 'total_trips')
                .take(10)
    )
    return t32_top10_trips


# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    window_spec = ps.window.Window.orderBy(pf.desc('average_unit_profit'))
    t32_top10_profit = (
        df_zones.withColumn('rank', pf.rank().over(window_spec))
                .select('rank', 'pickup_zone', 'average_unit_profit')
                .take(10)
    )
    return t32_top10_profit


# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    window_spec = ps.window.Window.orderBy(pf.desc('total_passengers'))
    t32_top10_passenger = (
        df_zones.withColumn('rank', pf.rank().over(window_spec))
                .select('rank', 'pickup_zone', 'total_passengers')
                .take(10)
    )
    return t32_top10_passenger

In [ ]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)
# print_count(zones)
# display(zones.take(10))

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)


In [ ]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

rank,pickup_zone,total_trips
1,Upper East Side South,122366
2,Upper East Side North,104192
3,Penn Station/Madison Sq West,97209
4,Midtown Center,94045
5,Murray Hill,88873
6,Midtown East,85651
7,Lincoln Square East,81298
8,Upper West Side South,75146
9,Clinton East,74960
10,Lenox Hill West,74693


In [ ]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

rank,pickup_zone,average_unit_profit
1,Bay Terrace/Fort Totten,311.9330683658483
2,Baisley Park,100.00592710440851
3,Prospect Park,90.58902850606826
4,Saint Michaels Cemetery/Woodside,66.14679218472433
5,Van Cortlandt Village,48.69059979610944
6,Grymes Hill/Clifton,38.65
7,Crotona Park,36.84437137879828
8,Flatbush/Ditmas Park,32.940319652777696
9,Eastchester,31.2081577880231
10,Flushing Meadows-Corona Park,29.53958083762334


In [ ]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

rank,pickup_zone,total_passengers
1,Upper East Side South,170668.0
2,Upper East Side North,144603.0
3,Penn Station/Madison Sq West,137864.0
4,Midtown Center,135530.0
5,Murray Hill,124433.0
6,Midtown East,120495.0
7,Lincoln Square East,115223.0
8,Clinton East,107811.0
9,Upper West Side South,106473.0
10,East Village,106252.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [ ]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'M'
DATA_FORMAT = 'delta'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: M
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-M-delta/
      Number of trips (dataset rows): 15,571,166
    


In [ ]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[71]: [[Row(rank=1, pickup_zone='Upper East Side South', total_trips=749245),
  Row(rank=2, pickup_zone='Upper East Side North', total_trips=704402),
  Row(rank=3, pickup_zone='Midtown Center', total_trips=474191),
  Row(rank=4, pickup_zone='Penn Station/Madison Sq West', total_trips=464982),
  Row(rank=5, pickup_zone='Lincoln Square East', total_trips=453356),
  Row(rank=6, pickup_zone='Upper West Side South', total_trips=451084),
  Row(rank=7, pickup_zone='Murray Hill', total_trips=446725),
  Row(rank=8, pickup_zone='Lenox Hill West', total_trips=444688),
  Row(rank=9, pickup_zone='Midtown East', total_trips=424265),
  Row(rank=10, pickup_zone='Union Sq', total_trips=381596)],
 [Row(rank=1, pickup_zone='Mariners Harbor', average_unit_profit=170.00196713252174),
  Row(rank=2, pickup_zone='Bay Terrace/Fort Totten', average_unit_profit=169.94730109365318),
  Row(rank=3, pickup_zone='East Flushing', average_unit_profit=116.0070934188661),
  Row(rank=4, pickup_zone='Rossville/Woodrow',

_Table 1. Pipeline performance for `parquet` format. (Time measured in seconds)_

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2,898,033 |  15,571,166 |  41,953,716 |  90,443,069 |  132,396,785 |
| execution time   (w/o 1.2)  | 8.94 | 104.4 | 840 | 1380 | 1626.6 |
| execution time              | 24.23 | 254.4 | 2221.2 | 2361 | 2751 |
| sec / 1M records (w/o 1.2)  | 3.08  | 6.7  | 20.02  | 15.25  | 12.28  |
| sec / 1M records            | 8.36  | 16.33  | 52.94  | 26.10  | 20.78  |

_Table 2. Pipeline performance for `delta` format. (Time measured in seconds)_

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2,898,033 |  15,571,166 |  41,953,716 |  90,443,069 |  132,396,785 |
| execution time   (w/o 1.2)  | 5.79 | 8.65 | 9.13 | 10.53 | 12.21 |
| execution time              | 11.29 | 17.05 | 18.52 | 20.96 | 21.67 |
| sec / 1M records (w/o 1.2)  | 1.99  | 0.55  | 0.21  | 0.11  | 0.09  |
| sec / 1M records            | 3.89  | 1.09  | 0.44  | 0.23  | 0.16  |